#### Combine a rule-based and a machine-learning chatbot (like those of the example code),using rules and training data of your choice (you may retain the example setup if you wish) to implement a bot that answers clear-cut situations based on rules and defaults to the responses from a fitted neural network when no rule matches. Interact with it and discuss the strengths and the weaknesses of your bot. Please include code snippets and an example dialogue in your response.

In [ ]:
import json
 
# Data to be written
intents = {
    'intents':[{'tag': 'greeting',
                'patterns': ['Hi', 'Hello', 'Howdy', 'How do you do', 'Hey', 'Hi there', 'Good Morning',
                             'Good Afternoon', 'Good evening', 'Yo, what\'s up', 'How is it going', 'How are you doing',
                             'How goes it','How\'s life','Sup', 'Whazzup', 'G\'day mate', 'Hiya'], 
                'responses':['Hello', 'Hi', 'Hi Hi', 'Good Morning',
                             'Good Afternoon', 'Good evening', 'No much and you', 'Life is amazing', 'Good, thank you, and you', 
                             'Meh', 'Nah', 'Wonderful',
                             'Enchanting', 'It\'s so amazing that you can\'t even imagine', 'G\'day mate', 'Hiya','A big HELLO to you too']},
               {'tag': 'goodbye',
               'patterns': ['Bye', 'Good bye', 'Ciao', 'Ciao ciao', 'auf Wiedersehen','See you later', 'See you soon', 'Hope to see you soon',
                            'Talk to you later', 'Gotta go','Take it easy','I\'m off', 'Have a nice one', 'Have a good day', 'Good Day',
                            'Good Day Sweetie', 'Have a nice day', 'I look forward to chatting with you again', 'Until then', 'Take care', 
                            'Good night', 'Later', 'Laters', 'Catch you later', 'Peace', 'Peace out', 'I\'m out', 'I gotta jet ', 
                            'I gotta take off', 'I gotta hit the road', 'I gotta head out','Sayonara', 'Arrivederci', 'Adios' ],
                'responses':['See you later', 'Have a nice day', 'Sweet Dreams', 'Bye' 'Good bye', 'Come back again', 'Same here', 
                             'Peace', 'Peace out',
                             'Sayonara', 'Arrivederci', 'Adios','Good Day Sweetie',  'Have a nice one', 'Have a good day', 
                             'Good Day', 'May the force be with you',
                             'يلاباي', 'Ciao','Ciao ciao','May god be with you ','ttyl', 'May our paths cross again','Bye bye', 
                             'Bye for now','Until the next time',
                             'Bon vent', 'À la revoyure']},
               {'tag':'thanks',
                'patterns':['Thanks', 'Thank you', 'Thank you very much', 'That\'s very helpful', 'That\'s very helpful', 'That\'s very insightful',
                            'Thank you for the help', 'Thanks for the help', 'Danke', 'Amazing', 'Much obliged', 'Thanks for having my back', 'Cheers',
                            'Many thanks', 'Thx', 'I am so grateful', 'Highly appreciate it', 'I could not have done it without you', 'I owe you one',
                            'I owe you a big one', 'Please accept my deepest gratitude', 'This has been such a blessing', 'I can\'t thank you enough', 
                            'That\'s so kind of you', 'You are too kind', 'Thank you for taking your time to help me', 'Your support means the word', 
                            'Your support means the world to me','Mille Grazie', 'Merci', 'I sncerely applaud you', 'I am forever indebted', 
                            'I am blown away by your kindness', 'I want to acknowledge how much you have done', 'Thanks in advance', 'I stand in recognition', 
                            'I have to give credit where it is due'],
                'responses':['Happy to help!', 'Any time!', 'My pleasure', 'You are most welcome!', 'The pleasure is mine!','No Biggie', 'No problem',
                             'I am glad to help you in anyway I can', 'I am glad', 'Cheers', 'Wunderbar!', 'You are welcome', 'Amazing']},
               {'tag':'about',
                'patterns':['Who are you?', 'What are you?','Who is this?', 'Who am I chatting with', 'With whom am I chatting?', 'To whom am I speaking?',
                            'Who might you be?', 'Are you a human?', 'Are you a chatbot?', 'What is your name?', 'How may I address you?', 'How should I call you?',
                            'Where am I?'],
                'responses':['Oh hello there, I am Alexandria, your bot assistant!', 'Hello, I am David, your loyal Artificial Intelligent servant!', 'Call me Betty!', 
                             'You are here at our automated customer service, I am John, your AI bot assistant', 'I am not human, I am Alexandria, your bot assistant', 
                             'Call me Johan!', 'My name is Hrafn!']},
               {'tag': 'help',
                'patterns':['Could you help me?', 'Could you help me with something?', 'Can you help me with something?', 'Give me a hand please', 
                            'Can you help?', 'What can you do for me?', 'I need a support', 'I desprerately need your help', 'Support me please!', 
                            'Oh Gosh, I need your help for something!', 'Help, help!', 'Can you do something for me, please?'],
                'responses':["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you",'How may I be of service?',
                             'What do you want me to do?', 'Pray!', 'Pray tell','What\'s up?', 'Yes, what?', 'Yes, sure', 'Do tell, please!', 'Ok, what type of services do you require?',
                             'Yep, what\'s the matter?', 'Gladly, do tell!', 'It will be my pleasure!', 'I always have time for you, my liege!', 'Alright, what do you need?', 
                             'What is it?', 'Ok', 'Okay', 'Sure', 'Certainly', 'undoubtedly!','Unquestionably!', 'Yaaaas!']},
               {"tag": "createaccount",
                "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", 
                             "can you create an account for me", "how to open a new account"],
                "responses": ["You can just easily create a new account from our web site",
                              "Just go to our web site and follow the guidelines to create a new account"]},
               {'tag':'billing',
                'patterns':['Can you check my last payment,please?', 'Why is my payment not going through?', 'What is wrong with your billing system?', 
                            'I am having difficulties to make a payment', 'Can I make a payment here?', 'Can I make a payment through your website?',
                            'Refund please!', 'Can I make a payment with my personal cheque?','My credit card is expired, I would like to update my billing info!',
                            'There is a mistake on my bill for this month!', 'I have been over charged!', 'I demande a refund!!!', 'I would like to cancel my subscription, please!', 
                            'For some reason, I have not received this month\'s bill yet, What\'s going on?', 'Where can I make a payment, through my financial institute or by phone?', 
                            'I can\'t understand my bill, I need some explaination, please!', 'Can you help me to check my current balance, please?'],
                'responses':['I am so sorry to hear that. Let me check it out for you.', 'I understand totaly, I will help you in anyway I can.',
                             'I am sorry for the inconvenience, I will try to sort it out for you.', 'No problem, I will pull your info at once.', 
                             'No problem, right away!', 'Certainly, I will looking into it immediately!', 'For the refund, I will direct you to the billing department. A moment please.', 
                             'A moment please.', 'Right away!', 'Of course.', 'Absolutely', 'For more info, please dial our tolfree number at 1-800-999-9999.',
                             'I am sorry, I can\'t process your info at this time, please contact our customer service at 1-888-9999-9999.', 
                             'Sorry, I can\'t help you with your problem, you must contact your finacal institute for more information.', 
                             'Sorry, your credit card has been declined. Perhaps, you can try to process your payment with a different credit card.', 
                             'It is very unfortunate that our system is down at the moment, please contact us again at a later moment.', 
                             'I am sorry, but we do not accept Union Pay!', 'Your current balance is 465 dollars, and your billing cycle is from 2022-10-15 to 2022-11-15.', 
                             'You have a due of 115 dollars, would you like to make a payment right now?']},
               {"tag": "complaint",
                "patterns": ["I have a complaint? ", "I want to raise a complaint", "I would like to make a complaint about your service",
                             'I would like to get in touch with a manager regarding a complaint, please!', 'Blimey, I am here to file a complaint.', 
                             'WTF is wrong with your service, it\'s disgusting. I need to make a complaint!', 'You guys are a fucking piece of shit, I want my money back!', 
                             'Horrible service, I would like to file a compalint.', 'I would like to speak to a manager immediately!!!', 'Where is my freaking product?',
                             'Get me someone who is not incompetent, immediatly!', 'Fuck you!', 'A piece of shit!', 'You guys are a bunch of thieves!', 'You\'ve lost my package!', 
                             'I am so freaking pissed, What the fuck is going on with my order?', 'Freaking hell, I have been waiting on the phone for 45 minutes!'],
                "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused",
                              'We are very sorry to hear that, we will assist you immediately.', 'I will direct your complaint to our customer satisfaction department right away.',
                              'You are our priority, a moment please!', 'I will direct you to our shipping department, please wait.', 'We do not tolerate profanity spiel, please try it again!',
                              'What is the subject of your complaint?', 'To speak to a manager in charge, please dial 1-880-666-6666.', 'Thank your for your patience, our shipping department is trying to locate your order.',
                              'Thank your for your patience. Your order is at the custom, please visit https://www.cbsa-asfc.gc.ca/menu-eng.html for more details.']}]}
               



In [ ]:
with open('json_data.json', 'w') as outfile:
    json.dump(intents, outfile, indent=4)   # <-- change new_intents to intents

with open('json_data.json') as json_file:
    data = json.load(json_file)

print(data['intents'])


[{'tag': 'greeting', 'patterns': ['Hi', 'Hello', 'Howdy', 'How do you do', 'Hey', 'Hi there', 'Good Morning', 'Good Afternoon', 'Good evening', "Yo, what's up", 'How is it going', 'How are you doing', 'How goes it', "How's life", 'Sup', 'Whazzup', "G'day mate", 'Hiya'], 'responses': ['Hello', 'Hi', 'Hi Hi', 'Good Morning', 'Good Afternoon', 'Good evening', 'No much and you', 'Life is amazing', 'Good, thank you, and you', 'Meh', 'Nah', 'Wonderful', 'Enchanting', "It's so amazing that you can't even imagine", "G'day mate", 'Hiya', 'A big HELLO to you too']}, {'tag': 'goodbye', 'patterns': ['Bye', 'Good bye', 'Ciao', 'Ciao ciao', 'auf Wiedersehen', 'See you later', 'See you soon', 'Hope to see you soon', 'Talk to you later', 'Gotta go', 'Take it easy', "I'm off", 'Have a nice one', 'Have a good day', 'Good Day', 'Good Day Sweetie', 'Have a nice day', 'I look forward to chatting with you again', 'Until then', 'Take care', 'Good night', 'Later', 'Laters', 'Catch you later', 'Peace', 'Peace 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
for intent in data['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])    



In [ ]:
words[:10]

['Hi', 'Hello', 'Howdy', 'How', 'do', 'you', 'do', 'Hey', 'Hi', 'there']

In [ ]:
classes[:10]

['greeting',
 'goodbye',
 'thanks',
 'about',
 'help',
 'createaccount',
 'billing',
 'complaint']

In [ ]:
documents[:10]

[(['Hi'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Howdy'], 'greeting'),
 (['How', 'do', 'you', 'do'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hi', 'there'], 'greeting'),
 (['Good', 'Morning'], 'greeting'),
 (['Good', 'Afternoon'], 'greeting'),
 (['Good', 'evening'], 'greeting'),
 (['Yo', ',', 'what', "'s", 'up'], 'greeting')]

In [ ]:
nltk.download('omw-1.4')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


153 documents
8 classes ['about', 'billing', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'thanks']
251 unique lemmatized words ["'m", "'s", "'ve", ',', '.', '45', 'a', 'about', 'accept', 'account', 'acknowledge', 'address', 'adios', 'advance', 'afternoon', 'again', 'am', 'amazing', 'an', 'applaud', 'appreciate', 'are', 'arrivederci', 'auf', 'away', 'back', 'balance', 'be', 'been', 'big', 'bill', 'billing', 'blessing', 'blimey', 'blown', 'bunch', 'by', 'bye', 'ca', 'call', 'can', 'cancel', 'card', 'care', 'catch', 'charged', 'chatbot', 'chatting', 'check', 'cheer', 'cheque', 'ciao', 'compalint', 'complaint', 'could', 'create', 'credit', 'current', 'danke', 'day', 'deepest', 'demande', 'desprerately', 'difficulty', 'disgusting', 'do', 'doing', 'done', 'due', 'easy', 'enough', 'evening', 'expired', 'explaination', 'file', 'financial', 'for', 'forever', 'forward', 'freaking', 'fuck', 'fucking', "g'day", 'get', 'give', 'go', 'going', 'good', 'gosh', 'got', 'grateful', 'grat

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
31/31 [==============================] - 1s 2ms/step - loss: 2.0209 - accuracy: 0.2418
Epoch 2/200
31/31 [==============================] - 0s 2ms/step - loss: 1.9298 - accuracy: 0.2614
Epoch 3/200
31/31 [==============================] - 0s 2ms/step - loss: 1.7693 - accuracy: 0.3399
Epoch 4/200
31/31 [==============================] - 0s 2ms/step - loss: 1.6445 - accuracy: 0.4183
Epoch 5/200
31/31 [==============================] - 0s 2ms/step - loss: 1.5145 - accuracy: 0.4641
Epoch 6/200
31/31 [==============================] - 0s 2ms/step - loss: 1.3390 - accuracy: 0.5294
Epoch 7/200
31/31 [==============================] - 0s 2ms/step - loss: 1.1876 - accuracy: 0.5621
Epoch 8/200
31/31 [==============================] - 0s 2ms/step - loss: 1.0571 - accuracy: 0.6536
Epoch 9/200
31/31 [==============================] - 0s 2ms/step - loss: 0.8009 - accuracy: 0.7124
Epoch 10/200
31/31 [==============================] - 0s 2ms/step - loss: 0.7332 - accuracy: 0.7778
Epoch 11/

In [25]:
userinput = input('Write here to what you want the bot to respond: ')

Write here to what you want the bot to respond: A piece of shit! 


In [26]:
uw = set([ lemmatizer.lemmatize(w) for w in nltk.word_tokenize(userinput) ])
BoW = [ 1 * (w in uw) for w in words ]
print(len(BoW))
threshold = 0.3 # discard unpromising results
import tensorflow as tf
datapoint = tf.convert_to_tensor(np.array([BoW]))
p = model.predict(datapoint)[0] # predict just this one data point
print(classes) # reminder
for outcome, label in zip(p, classes):
  if outcome > threshold: # could be a match
    print(outcome, label)

251
1/1 [==============================] - 0s 17ms/step
['about', 'billing', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'thanks']
1.0 complaint


In [38]:
best = np.argmax(p) # use the highest as our match
chosen = classes[best] # the class to use to respond
print(f'Chose class {chosen}')
from random import choice # we pick one at random
for i in intents['intents']:
  if i['tag'] == chosen: 
    print(choice(i['responses']))
    break

Chose class complaint
We do not tolerate profanity spiel, please try it again!
